# <center >OpenAI Agents SDK + MCP 智能体开发实战</center>
## <center>Part 2. MCP 接入方法与Hooks事件回调实战</center>

&emsp;&emsp;在上一小节课程中，我们重点介绍了`OpenAI Agents SDK`框架运行的底层原理及完整的生命周期，针对其核心组件`Agent`和`Handoffs`展开了详细的介绍和实践，同时对构建代理运行时的`Runner`运行机制做了重点说明。截止到目前相信大家已经对`Agent`和`Runner`的运行机制有了整体的认识，并对必要的一些参数配置有了深入的了解。

&emsp;&emsp;快速回顾上节课程中讲解的内容。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091334117.png" width=60%></div>

&emsp;&emsp;首先，构建`Agent`组件对象时可用的参数配置如下所示：（绿色为上节课程中已经重点介绍过的参数）

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=green>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=green>**handoffs**</font>         | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>       | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| `mcp_servers`       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| `mcp_config`        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

</div>

&emsp;&emsp;同时，对应的`Runner`组件对象时可用的参数配置如下所示：（绿色为上节课程中已经重点介绍过的参数）

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 组件核心参数</font></p>
<div class="center">


| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**starting_agent**</font>        | 要运行的起始代理。                                                                                        |
|  <font color=green>**input**</font>                     | 代理的初始输入。可以传递一个用户消息的字符串，或一个输入项的列表。                                           |
| <font color=green>context<font color=red>                | 运行代理时使用的上下文。                                                                                 |
| `max_turns`             | 运行代理的最大回合数。回合定义为一次 AI 调用（包括可能发生的任何工具调用）。                                   |
| `hooks`                  | 接收各种生命周期事件回调的对象。                                                                         |
| `run_config`             | 整个代理运行的全局设置。                                                                                 |
| `previous_response_id`   | 上一个响应的 ID，如果使用 OpenAI 模型通过 Responses API，这允许你跳过传递上一个回合的输入。                   |
</div>


&emsp;&emsp;其中暂时未详细讲解的参数分别对应的是`OpenAI Agents SDK`框架中接入`MCP`服务器、输入\输出安全护栏、事件回调三个功能组件的配置。因此本节课程我们就进一步介绍如何将`MCP`服务器接入到`Agent`运行时中，并介绍如何正确构建事件回调的`Hooks`。

&emsp;&emsp;首先，我们来看一下如何将`MCP`服务器接入到`OpenAI Agents SDK` 框架中。

# 一、Agents SDK 与 MCP 的兼容情况

&emsp;&emsp;`OpenAI Agents SDK` 框架在`2025年3月27日`正式官宣支持`MCP`，即可以兼容任意基于`Model context protocol`协议开发的MCP服务器提供给`Agent`组件作为工具进行使用。 

&emsp;&emsp;`OpenAI Agents SDK`框架接入`MCP`服务器的方法并不复杂，仅需要在构建`Agent`对象时，像传递通过`@function_tool`装饰器定义的工具一样，将`MCP`服务器作为参数传递给`Agent`对象即可。只不过，不再通过`tools`参数，而是借助`mcp_servers`参数来传递。如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=green>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=green>**handoffs**</font>         | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>       | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| <font color=red>mcp_servers</font>       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| <font color=red>mcp_config</font>        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| `hooks`             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

&emsp;&emsp;这两个参数的含义如下所示：

- **mcp_servers**：该参数用于接收一个或多个`Model Context Protocol (MCP)`服务器的实例。当`Agent`运行时，它会自动加载`MCP`服务器，并获取`MCP`服务中定义的可用工具列表。
- **mcp_config**：该参数用于配置 `MCP` 服务器的相关设置。

&emsp;&emsp;其中对于`mcp_config`参数，从源码中看目前仅有一个`convert_schemas_to_strict`参数，该参数用于配置是否将`MCP`服务器的`schema`转换为严格模式。该参数类型是一个布尔值类型，默认为False。这个其实很好理解，`MCP (Model Context Protocol)` 是一个独立的协议，其 `Schema` 表示和 `OpenAI` 规范中 `Json Schema` 表示并不完全相同，当`convert_schemas_to_strict`设置为 `True` 时，会尝试转换（MCP Schema -> OpenAI Json Schema），如果转换失败，则捕获异常并记录日志，继续使用原始的 `schema`。对转换细节比较感兴趣同学可以在这里查看源码：https://github.com/openai/openai-agents-python/blob/main/src/agents/mcp/util.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141351816.png" width=60%></div>

&emsp;&emsp; 因此，如果想给`Agent`对象传递`MCP`服务器，其构建的代码形式如下所示：

```python
    mcp_agent=Agent(
        name="智能代理",
        instructions="可以调用MCP服务器中的工具来完成任务",
        mcp_servers=[mcp_server_1, mcp_server_2],  # server_1和server_2是MCP服务器的实例
        mcp_config={
            "convert_schemas_to_strict": False  # 默认是 False
        }
    )
```

&emsp;&emsp; 所以，接下来核心就是如何去构建`mcp_servers_1`和`mcp_servers_2`这样的`MCP`服务器实例。这里需要注意的是：`OpenAI Agents SDK`框架中兼容的是基于[Model Context Protocol](https://github.com/modelcontextprotocol)的开放协议规范，而该协议根据所支持和使用的传输协议不同，目前主要分为如下三种类型的服务器：

- **stdio 服务器**：作为应用程序的子进程运行。我们一般将这种类型的服务器视为“本地”运行。
- **http over sse 服务器**：作为独立的进程运行，并使用`sse`协议进行通信，可以通过`url`来访问。
- **StreamableHTTP 服务器**：可流式传输的`HTTP`传输,正在取代 `SSE`传输。

 &emsp;&emsp; 其中 `StreamableHTTP` 的支持刚刚于[modelcontextprotocolpython-sdk v1.8.0](https://github.com/modelcontextprotocol/python-sdk/releases/tag/v1.8.0)发布，所以`OpenAI Agents SDK`框架目前仅支持`stdio`和`http over sse`这两种类型。源码定义位置：https://github.com/openai/openai-agents-python/blob/main/src/agents/mcp/server.py#L24

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505121539603.png" width=60%></div>

&emsp;&emsp; 通过`MCP`的官方可以了解到：目前官方提供了`Python SDK`、`TypeScript SDK`、`Java SDK`、`Kotlin SDK`、`C# SDK`、`Swift SDK`，这意味着使用对应的`SDK`可以让开发者使用不同编程语言中实现`MCP`服务器或客户端：https://modelcontextprotocol.io/introduction

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141448279.png" width=60%></div>

 &emsp;&emsp; 自然，无论是自主研发`MCP`服务器，还是使用公开的`MCP`服务器，使用不同编程语言实现的`MCP`服务器，其接入时所依赖的底层环境和执行命令也会稍有不同，比如我们在接入`MCP`服务器时常见的一些命令如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>不同类型的MCP服务器接入指令汇总</font></p>
<div class="center">

| 编程语言/平台        | 启动命令示例                                        | 说明                                                 |                               |
| -------------- | --------------------------------------------- | -------------------------------------------------- | ----------------------------- |
| **Python**     | `python server.py` 或 `uv run server.py`       | 运行`Python SDK`版本的`MCP`服务器                            |                               |
| **TypeScript**    | `node server.js` 或 `npx -y server-package`    |  `TypeScript SDK` 是 `JavaScript` 的超集， 提供了完整的 `Node.js` 支持。    |                               |
| **kotlin**         | `java -jar server.jar`      | `Kotlin` 可以与 `Java` 无缝互操作，通常 `Kotlin` 编写的服务会被编译成 `.jar` 文件，与 `Java` 代码相同。                               |                               |
| **C#**  | `dotnet run` 或 `dotnet build` 后运行生成的可执行文件     | 运行 `C# SDK`版本的 `MCP` 服务器。                         |                               |
| **Java**       | `java -jar server.jar`                        | 运行 `Java SDK`版本的 `MCP` 服务器。                         |                               |
| **Swift** | `swift run server.swift`| 运行 `Swift SDK` 版本的 `MCP` 服务器 |


</div>


&emsp;&emsp;不同的语言编写的 `MCP` 服务器需要使用对应的环境来运行。例如，`Python` 服务器需要 `Python` 环境，`TypeScript` 服务器需要 `Node.js` 环境。除此以外，虽然`Go` 和 `Rust` 没有官方 `SDK`，但由于这些语言在高性能系统中的广泛使用，社区和第三方开发者也发布了很多`Go` 和 `Rust` 版本的`MCP`服务器，在使用的时候，按照`MCP`服务器发布源的说明进行配置即可。

&emsp;&emsp; 以上几种不同类型的`MCP`服务器，无论是自主研发还是直接使用公开的`MCP`服务器，在`OpenAI Agents SDK`框架中，可以使用`MCPServerStdio`和`MCPServerSse`分别实例化`stdio`和`http over sse`类型的`MCP`服务器。

&emsp;&emsp;本节课我们首先来看`MCPServerStdio`类型的`MCP`服务器。

# 二、Stdio MCP 服务器接入实战

&emsp;&emsp; 在`OpenAI Agents SDK`框架中，基于`Stdio` 传输协议的`MCP`服务器需要通过 `MCPServerStdio`类来构建，根据其源码定义，我们首先可以梳理出`MCPServerStdio`类能够接收的参数如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerStdio可接收参数</font></p>
<div class="center">



| 参数名称                          | 类型                          | 默认值 | 描述                                                                                                                                                                                                 |
|-----------------------------------|-------------------------------|--------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `params`                          | `MCPServerStdioParams`        | -      | 配置服务器的参数，包括启动服务器的命令、传递给命令的参数、设置的环境变量、用于生成进程的工作目录以及发送/接收消息时使用的文本编码。                                                                 |
| `cache_tools_list`               | `bool`                        | `False`| 是否缓存工具列表。如果为 `True`，工具列表将被缓存，仅在第一次从服务器获取。如果为 `False`，每次调用 `list_tools()` 时都会从服务器获取工具列表。缓存可以通过调用 `invalidate_tools_cache()` 来失效。 |
| `name`                            | str 或 None               | `None` | 服务器的可读名称。如果未提供，将根据命令自动创建名称。                                                                                                                                              |
| `client_session_timeout_seconds`  | float 或 None                | `5`    | 传递给 MCP `ClientSession` 的读取超时时间时。                                                                                                                                                             |

&emsp;&emsp; 对`cache_tools_list`参数来说，当设置为 `True` 时，工具列表只会从服务器获取一次并缓存起来，在之后调用 `list_tools()` 方法时会直接使用缓存，而不会每次都向服务器发请求。这个机制可以提高性能和减少延迟，因为如果频繁调用 `list_tools()`而没有缓存，就会导致每次都实际向`MCP`服务器发起请求，从而增加延迟。所以该参数适用的场景是：当能够确定服务器的工具列表不会改变时使用，设置为`True`，而如果服务器工具列表可能会动态变化，则使用其默认参数，设为 `False`。`client_session_timeout_seconds` 参数则非常容易理解，其用于设置`client_session`的超时时间，控制在等待服务器响应时最长等待多久。如果服务器在指定时间内没有响应，操作会超时。默认值为5秒。大家可以根据实际情况设置。


&emsp;&emsp;最后，需要重点关注的是`params`参数，该参数就是配置`MCP`服务器启动的命令，认证参数等配置信息，该参数是一个字典，字典中可包含如下参数设置：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>MCPServerStdio中 Params 参数设置</font></p>
<div class="center">

| 参数                     | 类型                     | 必填 | 默认值 | 说明                                                                                     |
|------------------------|------------------------|------|--------|----------------------------------------------------------------------------------------|
| `command`              | `str`                  | 是   | 无     | 启动服务器的可执行文件。例如：`"python"` 或 `"node"`。                                   |
| `args`                 | `list[str]`           | 否   | `[]`   | 传递给可执行文件的命令行参数。例如：`["foo.py"]` 或 `["server.js", "--port", "8080"]`。 |
| `env`                  | `dict[str, str]`      | 否   | 无     | 为服务器进程设置的环境变量。例如：`{"api_key": "xxxx"}`。                       |
| `cwd`                  | `str、path`        | 否   | 无     | 启动进程时使用的工作目录，指定服务器进程的当前工作路径，以便访问相对路径的文件。     |
| `encoding`             | `str`                  | 否   | `"utf-8"` | 与服务器通信时使用的文本编码。例如：`"utf-8"`、`"ascii"`。                             |
| `encoding_error_handler` | `Literal["strict", "ignore", "replace"]` | 否 | `"strict"` | 文本编码错误处理方式。`"strict"` 遇到错误时抛出异常；`"ignore"` 忽略错误字符；`"replace"` 用替代字符替换错误字符。 |

&emsp;&emsp;这里需要额外扩展了解的参数是`cwd`参数，这个参数指的是接入的`MCP`服务器如果需要依赖于特定目录下的文件或资源时，可以通过该参数来指定服务器进程的当前工作目录。或者需要在特定目录中输出文件时，也可以通过该参数来指定。

&emsp;&emsp;接下来，我们就开始尝试进行`MCP`服务器的接入测试。

## 2.1 接入公开MCP服务器

&emsp;&emsp;关于公开的`MCP`服务器，有如下几个主要的`MCP`导航网站，可以快速查询到不同语言、不同类型及不同场景的`MCP`服务器，大家可以参考这些网站，了解并选择适合实际需求的`MCP`服务器。如下所示：

- Model Context Protocol servers: https://github.com/modelcontextprotocol/servers

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141603229.png" width=60%></div>

- Awesome MCP Servers: https://github.com/punkpeye/awesome-mcp-servers

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141603230.png" width=60%></div>

- MCP导航： https://mcp.so/

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141603231.png" width=60%></div>

- Smithery：https://smithery.ai/

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141603232.png" width=60%></div>

&emsp;&emsp; 接下来，我们将以官方支持的`MCP`服务器为例，接入一个`Filesystem MCP Server`。该服务器支持基本的文件操作，如读取、写入和删除文件，并支持文件搜索和列出目录内容等功能。详细的工具函数可以参考其官方文档：：https://github.com/modelcontextprotocol/servers/tree/main/src/ , 同时需要注意的是，接入`MCP`工具时，我们需要使用异步运行和上下文管理。因此，建议直接使用`.py`文件来运行，而非`Jupyter Notebook`，因为`Jupyter Notebook`中存在一些异步执行的限制问题。相关的代码文件名称为：`1_filesystem_mcp.py`,大家可以在网盘中下载后运行使用。

&emsp;&emsp;我们依然使用`DeepSeek`的`API`来运行（注意：要新建`.env`文件并配置`DeepSeek`的参数）这里需要关注的点是：

1. 当接入公开的`MCP`服务器时，一定要按照官方的说明进行配置，比如我们所接入的`Filesystem MCP Server` ，其官方说明只允许在指定的目录中操作文件，因此，我们需要在构建`MCP`服务器时，指定一个具体的目录。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141645512.png" width=60%></div>

2. 确定所选择的`MCP`服务器是基于哪种类型的`SDK`构建的，要确保当前运行环境下有对应的依赖环境，比如我们所接入的`Filesystem MCP Server` ，其基于`TypeScript`构建，所以需要确保当前环境下有`Node.js`环境。 以`Windows`开发环境为例，在 `PowerShell` 中输入 `node -v` 命令，如果返回版本信息，则说明当前环境有`Node.js`环境，否则需要安装`Node.js`。安装教程可以参考：https://nodejs.org/en/download/package-manager 。通过`Get-Command` 来查看 `npx` 的完整路径:

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141655821.png" width=60%></div>

&emsp;&emsp;然后在`MCPServerStdio`的`params`参数中，通过`command`参数来指定`npx`的完整路径，这在`windows`开发环境中非常关键，因为大部分`Windows`电脑环境即使配置了全局的环境变量，但是在加载`MCP`服务器时，依然会报错。填写完整的`npx`路径，可以避免这个问题。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141700304.png" width=60%></div>

&emsp;&emsp; 3. 在运行`MCP` 服务器时，由于 `Stdio` 传输机制通过子进程在本地环境中执行，接入 `Runner` 后多层子进程的嵌套可能导致 `Python` 的垃圾回收机制异常，进而引发以下错误：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141710265.png" width=60%></div>

&emsp;&emsp;解决的办法是：提前销毁 `MCPServerStdio` 对象并强制 `GC` 在主循环结束前执行，即在`async with MCPServerStdio(...)` 块退出后立即清空并强制 GC，可有效避免。（注意：此情况仅在`Windows`开发环境，`Mac`和`Linux`环境不存在该问题）

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141713464.png" width=60%></div>

&emsp;&emsp; 了解并完成以上配置后，即可直接运行`1_filesystem_mcp.py`文件进行`MCP`接入测试。 这里需要注意：运行该程序需要在`Python`虚拟运行环境下，并正确安装`pip install openai openai-agents` 依赖包即可。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141731273.png" width=60%></div>

## 2.2 接入自定义MCP服务器

&emsp;&emsp;关于如何构建自定义的`MCP`服务器，在《【加餐】OpenAI Agents SDK 开发实战 - 11.Agents SDK接入本地MCP服务器流程》 小节已经详细的做了介绍，我们这里直接用来测试，其`MCP`构建的流程依次是：

1. 使用 `uv` 创建虚拟环境;
2. 创建服务器，指定 `OpenWeather API KEY`；
3. 创建 `.env` 文件，存储调用模型的`baseurl`等信息；
4. 创建客户端，用来接收用户的输入，实际的查询天气信息并返回给用户；
5. 最后，使用命令`uv run client.py server.py`启动项目。

&emsp;&emsp;相关的代码文件夹为：`GetWeatherMCP_client`,大家可以在网盘中下载后运行使用。这里我们直接运行测试：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141739990.png" width=60%></div>

&emsp;&emsp;而对于`OpenAI Agents SDK`框架来说，其`MCPServerStdio`本质上是替代了我们手动编写的`server.py`，因此是可以直接使用`MCPServerStdio`来构建`MCP`服务器并接入到`Agent`运行时中的，这里我们可以通过`2_getweather_mcp.py`文件来测试。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141757147.png" width=60%></div>

## 1.3 构建 Multi-MCP Agent

&emsp;&emsp;在`OpenAI Agents SDK`框架中，我们可以同时接入多个`MCP`服务器，并让`Agent`根据用户的问题灵活选择合适的工具，并给出详细的回答。接入的方法并不复杂，只需要在`mcp_servers`参数中传入一个包含多个`MCP`服务器实例的列表即可。这里大家可以参考`3_multi_mcp.py`文件来测试。

&emsp;&emsp;除了代码的实现外，需要额外强调的一个优化细节是：对于多`MCP`服务器接入，这里有一点需要：每个`MCP`服务器其实都有自己的安全边界和上下文，特别是当`MCP Server`需要依赖某个外部环境时，比如文件系统路径、数据库连接等，在这段示例中给大家复现一个问题：

- 当我们先查询天气再写入文件时，文件系统`MCP`服务器总是找不到写入文件的有效路径；
- 当我们先创建文件再查询天气时，文件已经在合法的文件系统目录中存在，所以后续的写入操作是被允许的；

&emsp;&emsp;我们现在使用的`instructions`是这样的：

```python
    # 创建Agent并注册两个MCP服务器
    agent = Agent(
        name="MultiToolAgent",
        instructions="""你是一个多功能助手，同时具备文件系统操作和天气查询能力。
                        当用户询问文件相关问题时，使用文件系统工具读取、搜索或操作文件。
                        当用户询问天气相关问题时，使用天气工具查询特定城市的天气情况。
                        根据用户的问题灵活选择合适的工具，并给出详细的回答。

                        注意：查询天气时需要使用城市的英文名称""",
        model=OpenAIChatCompletionsModel(
            model=os.getenv("DEEPSEEK_MODEL"),
            openai_client=deepseek_client,
        ),
        mcp_servers=[fs, weather]  # 注册两个MCP服务器
    )
```

&emsp;&emsp;其运行结果如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141856316.png" width=60%></div>

&emsp;&emsp;针对这种情况，解决的办法就是去调整`Agent`的`instructions`参数，明确的指定文件操作的限制和正确的文件操作路径。这对一些依赖本地文件系统的`MCP`服务器来说非常有用。比如我们修改提示词如下：

```python
        instructions="""你是一个多功能助手，同时具备文件系统操作和天气查询能力。

        当操作文件时，请注意：
        1. 只能在 filesystem_files 目录下创建和修改文件
        2. 始终使用完整路径，例如 filesystem_files/filename.txt
        3. 在写入文件之前确保文件已存在

        当查询天气时，请使用城市的英文名称。

        当需要将天气信息保存到文件时，请遵循以下顺序：
        1. 首先创建或确认文件存在
        2. 然后查询天气信息
        3. 最后将信息写入文件

        根据用户的问题灵活选择合适的工具，并给出详细的回答。
        """,
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505141904418.png" width=60%></div>

&emsp;&emsp;在不修改其他代码逻辑的情况下，再次进行调用测试就能发现其已经可以正常调用不同的`MCP`服务器并顺利执行。如果大家在后续进行开发时涉及到一些特定路径、目录等`MCP`服务器的配置，建议采用这种优化思路进行`Prompt`的调整，可以稳定提升任务执行的准确性。

# 三、Agents SDK 中的事件回调

&emsp;&emsp;在`OpenAI Agents SDK`框架中，事件回调是`Agent`运行时的一个重要组成部分，通过回调函数，开发者可以捕获和处理`Agent`运行时的各种事件，如工具调用、消息传递等。正如我们上面的开发的案例，可以通过`print`函数打印一些`Agent`运行过程的数据，但是更加精细化的数据，比如每一步解析出了哪些 `MCP/Tool`的参数，执行了哪些工具，工具的返回结果是什么，这些信息很难通过`print`函数来获取。


&emsp;&emsp;`OpenAI Agents SDK`的完整生命周期我们在上节课程中给大家介绍过，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505091334117.png" width=60%></div>

&emsp;&emsp;如上图所示的整个运行生命周期的各个阶段都可以通过回调函数来获取到（全部发生在`Runner` 事件后）。这里我们需要关注，在创建`Runner`对象时，有一个参数`hooks`，该参数就是用来接收自定义的回调函数。

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 组件核心参数</font></p>
<div class="center">


| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**starting_agent**</font>        | 要运行的起始代理。                                                                                        |
|  <font color=green>**input**</font>                     | 代理的初始输入。可以传递一个用户消息的字符串，或一个输入项的列表。                                           |
| <font color=green>**context**<font color=red>                | 运行代理时使用的上下文。                                                                                 |
| `max_turns`             | 运行代理的最大回合数。回合定义为一次 AI 调用（包括可能发生的任何工具调用）。                                   |
| <font color=red>hooks</font>                  | 接收各种生命周期事件回调的对象。                                                                         |
| `run_config`             | 整个代理运行的全局设置。                                                                                 |
| `previous_response_id`   | 上一个响应的 ID，如果使用 OpenAI 模型通过 Responses API，这允许你跳过传递上一个回合的输入。                   |
</div>


&emsp;&emsp;`OpenAI Agents SDK`框架中，提供了`RunHooks`类，该类中定义了多个回调函数，我们可以继承该类，并重写其中的回调函数，来自定义事件回调。其源码位置为：https://github.com/openai/openai-agents-python/blob/main/src/agents/lifecycle.py#L57

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151009990.png" width=60%></div>

&emsp;&emsp;`RunHooks`类中定义的回调函数基类中定义的回调方法如下所示：

- **on_agent_start**(context, agent)

&emsp;&emsp;该函数的触发时机是在`Agent`开始运行前，同时每次当前 `Agent` 变更时都会触发，其源码在`run.py`的744-753行可以找到，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151026267.png" width=60%></div>

&emsp;&emsp; 通过源码可以看到，`on_agent_start`回调函数中接收的参数是`Agent`对象和`RunContextWrapper`对象，其中`RunContextWrapper`对象，也就是我们上节课程中重点讲解的`OpenAI Agents SDK`框架中的运行上下文，因此，在该回调事件中，我们是可以获取到完整的`Agent`对象的属性，以及运行上下文中的数据。包括：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Hooks: on_agent_start</font></p>
<div class="center">

| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| `context.context`            | 上下文                    |
| `context.Usage.requests`     | 请求数                    |
| `context.Usage.input_tokens` | 输入 tokens               |
| `context.Usage.output_tokens`| 输出 tokens               |
| `context.Usage.total_tokens` | 总 tokens                 |
| `agent.name`                 | Agent 名称               |
| `agent.instructions`          | Agent 指令               |
| `agent.handoff_description`   | 交接代理的描述                  |
| `agent.handoffs`             | 交接的代理            |
| `agent.model`                | 使用的模型               |
| `agent.model_settings`       | 模型的配置                |
| `agent.tools`                | 注册的工具               |
| `agent.mcp_servers`          | MCP 服务器列表            |
| `agent.mcp_config`           | MCP 配置                  |
| `agent.input_guardrails`     | 输入护栏                  |
| `agent.output_guardrails`    | 输出护栏                  |
| `agent.output_type`          | 输出类型                  |
| `agent.hooks`                | 附加的钩子函数               |
| `agent.tool_use_behavior`    | 工具使用行为              |
| `agent.reset_tool_choice`    | 是否重置工具选择          |

</div>

&emsp;&emsp; 具体每个参数数据的提取方法，请参考`4_hooks_lifecycle.py`文件中的 `on_agent_start` 方法。

- **on_tool_start(context, agent, tool)**

&emsp;&emsp;该函数的触发时机是在工具被调用前，其源码位置在`_run_impl.py`的 `432-471` 行的 `run_single_tool` 函数中，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151138390.png" width=60%></div>

&emsp;&emsp; 通过源码看，对工具调用，除了正常接收`Agent`对象和`RunContextWrapper`对象外，还会接收一个`func_tool`对象，该对象就是我们所调用的工具对象，通过该对象，我们可以获取到工具的名称、参数等信息。因此，该回调函数中可以接收的数据如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Hook: on_tool_start</font></p>
<div class="center">

| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| `tool.name` | 工具名称 |
| `tool.description` | 工具描述 |
| `tool.params_json_schema` | 参数JSON模式 |
| `tool.inputSchema` | 输入模式 |
| `tool.strict_json_schema` | 严格JSON模式 |
| `context.context`            | 上下文                    |
| `context.Usage.requests`     | 请求数                    |
| `context.Usage.input_tokens` | 输入 tokens               |
| `context.Usage.output_tokens`| 输出 tokens               |
| `context.Usage.total_tokens` | 总 tokens                 |
| `agent.name`                 | Agent 名称               |
| `agent.instructions`          | Agent 指令               |
| `agent.handoff_description`   | 移交描述                  |
| `agent.handoffs`             | 可用的移交目标            |
| `agent.model`                | 使用的模型               |
| `agent.model_settings`       | 模型设置                  |
| `agent.tools`                | 注册的工具               |
| `tool.name`                  | 工具名称                 |
| `tool.description`           | 工具描述                 |
| `tool.params_json_schema`    | 参数JSON模式             |
| `tool.inputSchema`           | 输入模式                 |
| `tool.strict_json_schema`    | 严格JSON模式             |
| `agent.mcp_servers`          | MCP 服务器列表            |
| `agent.mcp_config`           | MCP 配置                  |
| `agent.input_guardrails`     | 输入护栏                  |
| `agent.output_guardrails`    | 输出护栏                  |
| `agent.output_type`          | 输出类型                  |
| `agent.hooks`                | 附加的钩子               |
| `agent.tool_use_behavior`    | 工具使用行为              |
| `agent.reset_tool_choice`    | 是否重置工具选择          |

</div>

&emsp;&emsp; 具体每个参数数据的提取方法，请参考`4_hooks_lifecycle.py`文件中的 `on_tool_start` 方法。

-  **on_tool_end(context, agent, tool, tool_result)**

&emsp;&emsp; 理解了`on_tool_start`回调函数后，再看`on_tool_end`回调函数就非常容易理解了。该回调函数在工具调用结束后触发，其源码位置同样在`_run_impl.py`中，其中 `449-455` 行的 `run_single_tool` 函数中，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151150801.png" width=60%></div>

&emsp;&emsp;因此该回调函数可以接收的参数如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Hook: on_tool_start</font></p>
<div class="center">

| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| `result` | 工具调用的返回结果 |
| `tool.name` | 工具名称 |
| `tool.description` | 工具描述 |
| `tool.params_json_schema` | 参数JSON模式 |
| `tool.inputSchema` | 输入模式 |
| `tool.strict_json_schema` | 严格JSON模式 |
| `context.context`            | 上下文                    |
| `context.Usage.requests`     | 请求数                    |
| `context.Usage.input_tokens` | 输入 tokens               |
| `context.Usage.output_tokens`| 输出 tokens               |
| `context.Usage.total_tokens` | 总 tokens                 |
| `agent.name`                 | Agent 名称               |
| `agent.instructions`          | Agent 指令               |
| `agent.handoff_description`   | 移交描述                  |
| `agent.handoffs`             | 可用的移交目标            |
| `agent.model`                | 使用的模型               |
| `agent.model_settings`       | 模型设置                  |
| `agent.tools`                | 注册的工具               |
| `tool.name`                  | 工具名称                 |
| `tool.description`           | 工具描述                 |
| `tool.params_json_schema`    | 参数JSON模式             |
| `tool.inputSchema`           | 输入模式                 |
| `tool.strict_json_schema`    | 严格JSON模式             |
| `agent.mcp_servers`          | MCP 服务器列表            |
| `agent.mcp_config`           | MCP 配置                  |
| `agent.input_guardrails`     | 输入护栏                  |
| `agent.output_guardrails`    | 输出护栏                  |
| `agent.output_type`          | 输出类型                  |
| `agent.hooks`                | 附加的钩子               |
| `agent.tool_use_behavior`    | 工具使用行为              |
| `agent.reset_tool_choice`    | 是否重置工具选择          |

</div>

&emsp;&emsp; 具体每个参数数据的提取方法，请参考`4_hooks_lifecycle.py`文件中的 `on_tool_end` 方法。

- **on_agent_end(context, agent, output)**

&emsp;&emsp; 该函数的触发时机是在`Agent`运行结束后，其源码位置在`run_impl.py`的 `670-684` 行的 `run_final_output_hooks` 函数中，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151222849.png" width=60%></div>

&emsp;&emsp;其中对于 `final_output`，如果定义了 `output_type`(在构建`Agent`对象时)，则输出为该类型的实例，否则为字符串，因此该参数可接收的数据如下：
<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Hook: on_tool_start</font></p>
<div class="center">

| 属性名                     | 描述                                                                                                   |
|--------------------------|--------------------------------------------------------------------------------------------------------|
| `output` | Agent最终返回的输出结果 |
| `context.context`            | 上下文                    |
| `context.Usage.requests`     | 请求数                    |
| `context.Usage.input_tokens` | 输入 tokens               |
| `context.Usage.output_tokens`| 输出 tokens               |
| `context.Usage.total_tokens` | 总 tokens                 |
| `agent.name`                 | Agent 名称               |
| `agent.instructions`          | Agent 指令               |
| `agent.handoff_description`   | 移交描述                  |
| `agent.handoffs`             | 可用的移交目标            |
| `agent.model`                | 使用的模型               |
| `agent.model_settings`       | 模型设置                  |
| `agent.tools`                | 注册的工具               |
| `tool.name`                  | 工具名称                 |
| `tool.description`           | 工具描述                 |
| `tool.params_json_schema`    | 参数JSON模式             |
| `tool.inputSchema`           | 输入模式                 |
| `tool.strict_json_schema`    | 严格JSON模式             |
| `agent.mcp_servers`          | MCP 服务器列表            |
| `agent.mcp_config`           | MCP 配置                  |
| `agent.input_guardrails`     | 输入护栏                  |
| `agent.output_guardrails`    | 输出护栏                  |
| `agent.output_type`          | 输出类型                  |
| `agent.hooks`                | 附加的钩子               |
| `agent.tool_use_behavior`    | 工具使用行为              |
| `agent.reset_tool_choice`    | 是否重置工具选择          |

</div>

- **on_handoff(context, from_agent, to_agent)**

&emsp;&emsp; 最后，Hook事件回调中，还有一个非常重要的回调函数，那就是`on_handoff`回调函数，该回调函数在`Agent`移交时触发，即当一个`Agent`将控制权移交给另一个`Agent`时触发。该回调函数的源码在`run_impl.py`的 `579-595`行，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505151309653.png" width=60%></div>

&emsp;&emsp;该回调函数可接收的数据如下所示：

- context：Runner 运行时的上下文数据；
- from_agent.name：移交控制权的 `Agent` 名称;
- to_agent.name：接收控制权的 `Agent` 名称;

&emsp;&emsp;所以，仅当构建的`Agent`对象有通过`handoffs`参数构建`Multi-Agent`系统时，才能够触发此回调函数。因此，对于`Multi-Agent`的复杂设计，我们在下一小节中课程中再结合实际的案例给大家展开详细的介绍。

&emsp;&emsp;除此以外，大家也能够发现在 `Agent` 对象中也有 `Hooks`参数：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Agent 组件核心参数</font></p>
<div class="center">

| 属性名                | 类型                                                                                          | 描述                                                                                                   |
|---------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| <font color=green>**name**</font>             | `str`                                                                                         | 代理的名称。                                                                                             |
| <font color=green>**instructions**</font>      | `str` \| `Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]` \| `None` | 代理的指令，用作“系统提示”。可以是字符串或动态生成指令的函数。                                           |
| <font color=green>**handoff_description**</font> | `str` \| `None`                                                                              | 代理的描述，用于代理作为交接时，让 LLM 知道它的功能和何时调用它。                                         |
| <font color=green>**handoffs**</font>         | list[Agent[Any] | Handoff[TContext]]                                                   | 代理可以委托的子代理列表。允许关注点分离和模块化。                                                       |
| <font color=green>**model**</font>              | `str` \| `Model` \| `None`                                                                   | 调用 LLM 时使用的模型实现。默认情况下，如果未设置，代理将使用 `openai_provider.DEFAULT_MODEL` 中配置的默认模型。 |
| <font color=green>**model_settings**</font>    | `ModelSettings`                                                                               | 配置模型特定的调优参数（例如温度、top_p）。                                                             |
| <font color=green>**tools**</font>       | `list[Tool]`                                                                                 | 代理可以使用的工具列表。                                                                                 |
| <font color=green>mcp_servers</font>       | `list[MCPServer]`                                                                             | 代理可以使用的模型上下文协议（MCP）服务器列表。                                                        |
| <font color=green>mcp_config</font>        | `MCPConfig`                                                                                   | MCP 服务器的配置。                                                                                       |
| `input_guardrails`  | `list[InputGuardrail[TContext]]`                                                             | 在代理执行之前并行运行的检查列表，仅在代理是链中的第一个代理时运行。                                     |
| `output_guardrails` | `list[OutputGuardrail[TContext]]`                                                            | 在生成响应后对代理的最终输出运行的检查列表，仅在代理生成最终输出时运行。                                 |
| `output_type`       | type[Any] | AgentOutputSchemaBase | None                                                 | 输出对象的类型。如果未提供，输出将为 `str`。                                                             |
| <font color=red>hooks</font>             | AgentHooks[TContext] | None                                                               | 接收代理生命周期事件回调的类。                                                                           |
| `tool_use_behavior` | Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction | 配置工具使用的处理方式。                                                                                 |
| `reset_tool_choice` | `bool`                                                                                       | 调用工具后是否将工具选择重置为默认值。默认为 `True`。确保代理不会进入工具使用的无限循环。                   |

</div>

&emsp;&emsp;而根据源码（https://github.com/openai/openai-agents-python/blob/main/src/agents/lifecycle.py#L57）， 其实是有`RunHooks`与 `AgentHooks` 两个基类，其中 `AgentHoos` 是用来管理单个`Agent`的生命周期（Multi-Agent我们可能仅关注重要的智能体），所以它们的区别就是：

- 作用域不同：`RunHooks`是全局作用域，应用于整个运行流程，传递给 `Runner.run()` 方法。而`AgentHooks`特定于单个 `Agent`，设置为 `agent.hooks` 属性；
- 方法名称不同：`RunHooks` 的方法带有 `on_agent_` 前缀，例如`on_agent_start`, `on_agent_end` 等。而`AgentHooks` 的方法没有 `agent` 前缀，如`on_start`, `on_end` 等;

&emsp;&emsp;源代码中两个回调是同时被调用的，即使用 `asyncio.gather()` 来并行执行。因此对于不同的情况， 所用的场景也不一样，当需要监控整个运行流程，包括多个代理之间的交互，则需要构建`RunHooks`，如果只关心特定代理的生命周期事件，则可以使用`AgentHooks`。

&emsp;&emsp;对于`AgentHooks`和`RunHooks`的结合使用，需要一个相对复杂的`Multi-Agent`架构场景来实现，同时每个`Agent`要包含各自独立的`Tool`或者`MCP`工具，同时，对于目前遗留的`Guardrail` 组件的使用方法，我们也需要结合实际的应用场景来做介绍，以上所有的内容，我们将在接下来的一节课程中，通过一个复杂的实际案例进行讲解。